In [2]:
import pandas as pd
import numpy as np
import time
import os
import datetime as dt
from functools import reduce
import pickle

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
#Betano_Stuff

leagues_betano = {
    'Championship':'Championship', 'LaLiga':'La_Liga',
    'Liga Europa - Jogos':'Liga_Europa','Liga dos Campeões - Jogos':'Champions',
    'MLS':'MLS','Premier League' : 'Premier_League',
    'Premier Liga':'RFPL','Premiership':'Premiership',
    'Primeira Liga':'Primeira_Liga','Série A': 'Serie_A'
}

def Betano_date(x):
    months = {
        'Janeiro':1,'Fevereiro':2,'Março':3,'Abril':4,'Maio':5,'Junho':6,
        'Julho':7,'Agosto':8,'Setembro':9,'Outubro':10,'Novembro':11,'Dezembro':12
    }
    x = x.split(', ')[1]
    day,month,year,time=x.split(' ')
    #date
    if month not in months:
        print(month)
        month = input('Qual é o mês?   ')
    else:
        month = months[month]
    date = dt.datetime.strptime(str(day)+'/'+str(month)+'/'+str(year),'%d/%m/%Y')
    #time
    h,m=time.split(':')
    H=int(h)*100
    time = H+int(m)
    return date,time

def Betano_league(x,leagues):
    if x not in leagues:
        print(x)
        raise('aa')
    else:
        return leagues[x]
    
    
def europe_filter(df):
    for i in range(len(df)):
        if (df.iloc[i]['League']=='Liga_Europa') or (df.iloc[i]['League']=='Champions'):
            for c in list(df.columns[8:-2]): #queremos o scrape date/time
                df.at[i,c]='None'
    return df


def Betano_nan(df): #remove os scrapes falhados
    lig=list(np.where(df['League'].isnull().values)[0])
    rest=list(np.where(np.logical_and(df['Date'].isnull().values,df['HomeTeam'].isnull().values,
                                     df['AwayTeam'].isnull().values))[0])
    dele = list(np.unique(np.array(lig+rest)))
    print(dele)
    df=df.drop(dele,axis=0).reset_index(drop=True)
    return df

def Betano_filter(df,l=leagues_betano):
    df = Betano_nan(df)
    Dates = [df.iloc[i]['Date'] for i in range(len(df))]
    date_time = list(map(Betano_date,Dates))
    Leag = [df.iloc[i]['League'] for i in range(len(df))]
    League = list(map(Betano_league,Leag,[l]*len(Leag)))
    Y25=[]
    N25=[]
    YBS=[]
    NBS=[]
    X1=[]
    _12=[]
    X2=[]
    for i in range(len(df)):
        #25
        if df.iloc[i]['Tag25']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_25+']]
            N25+=[df.iloc[i]['ODD_25-']]
        elif df.iloc[i]['Tag15']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_15+']]
            N25+=[df.iloc[i]['ODD_15-']]
        elif df.iloc[i]['Tag35']=='Mais de2.5':
            Y25+=[df.iloc[i]['ODD_35+']]
            N25+=[df.iloc[i]['ODD_35-']]
        else:
            if (df.iloc[i]['League']=='Liga Europa - Jogos') or (df.iloc[i]['League']=='Liga dos Campeões - Jogos'):
                Y25+=['None']
                N25+=['None']
            else:
                print(df.iloc[i]['League'],i)
                print('problema com 25')
                raise('aaa')
                
            
        #BS
        if df.iloc[i]['TagBS']=='Ambas Equipas Marcam (Sim/Não)':
            YBS+=[df.iloc[i]['ODD_BthScore_Yes']]
            NBS+=[df.iloc[i]['ODD_BthScore_No']]
        else:
            if (df.iloc[i]['League']=='Liga Europa - Jogos') or (df.iloc[i]['League']=='Liga dos Campeões - Jogos'):
                YBS+=['None']
                NBS+=['None']
            else:
                print(df.iloc[i]['League'],i)
                print('problema com BS')
                raise('aaa')
            
        #DC
        if df.iloc[i]['Tag1X']=='1X':
            X1+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='1X':
            X1+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='1X':
            X1+=[df.iloc[i]['ODD_X2']]
        else:
            X1+=['None']
        #--------------------------------------------------   
        if df.iloc[i]['Tag1X']=='12':
            _12+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='12':
            _12+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='12':
            _12+=[df.iloc[i]['ODD_X2']]
        else:
            _12+=['None']    
        #---------------------------------------------------
        if df.iloc[i]['Tag1X']=='2X':
            X2+=[df.iloc[i]['ODD_1X']]
        elif df.iloc[i]['Tag12']=='2X':
            X2+=[df.iloc[i]['ODD_12']]
        elif df.iloc[i]['TagX2']=='2X':
            X2+=[df.iloc[i]['ODD_X2']]
        else:
            X2+=['None']
            
    ODDH = list(df.iloc[:]['ODDH'])
    ODDD = list(df.iloc[:]['ODDD'])
    ODDA = list(df.iloc[:]['ODDA'])
    Date = [date_time[i][0] for i in range(len(date_time))]
    Time = [date_time[i][1] for i in range(len(date_time))]
    HT = list(df.iloc[:]['HomeTeam'])
    AT = list(df.iloc[:]['AwayTeam'])
    Scrape_Data = list(df.iloc[:]['Current_Date'])
    
    df = pd.DataFrame({
        'League':League,'Date':Date,'Time':Time,
        'HT':HT,'AT':AT,'ODDH':ODDH,'ODDD':ODDD,'ODDA':ODDA,
        'ODD_25+':Y25,'ODD_25-':N25,
        'ODD_BthScore_Yes':YBS,'ODD_BthScore_No':NBS,
        'ODD_1X':X1,'ODD_12':_12,'ODD_X2':X2
    })
    
    df = europe_filter(df)
    
    return df

def merge_Betano(w): #w é uma lista de dfs
#se encontrarmos o mesmo jogo, escolhemos a odd + baixa (da ODDH)
    df = pd.concat(w,sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    i=0
    while i<len(df):
        same = list(np.where(np.logical_and(
        np.logical_and(df['HT']==df.iloc[i]['HT'],df['AT']==df.iloc[i]['AT']),
        np.logical_and(np.array(list(map(lambda x:x.year,df['Date'])))==df.iloc[i]['Date'].year,
        np.array(list(map(lambda x:x.month,df['Date'])))==df.iloc[i]['Date'].month)))[0])
        if len(same)==1:
            i+=1
        else:
            ind=np.argmin([df.iloc[i]['ODDH'] for i in same])
            del same[ind]
            df = df.drop(same,axis=0).reset_index(drop=True)
            
    return df


def save_Betano(w):
    oo=os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data')
    i1 = oo[-1][-5]
    w.to_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano'+str(i1+1)+'.csv')

In [3]:
#Under_Stuff

In [2]:
#Sky_Stuff

In [25]:
def file_teams2(w1,w2,league): #w1 -> df Sky |||| w2 -> df Under
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('Teams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome Under]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        under_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
        under_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
    i = 0
    under_teams_copy = under_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(under_teams) and not(found):
                if sky_teams[ii]==under_teams[nn]:
                    same+=[[sky_teams[ii],under_teams[nn]]]
                    del under_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            under_teams_copy=under_teams.copy()
            same_copy=same.copy()
            t=0
            while under_teams!=[]:
                print('------')
                print(restos)
                ok=input(str(under_teams[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    under_teams=under_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,under_teams[t]],[under_teams[t],ok]]
                    del under_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(under_teams):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_under = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_under = [under_teams[i] for i in range(len(under_teams)) if (under_teams[i] not in same_under)]
        missing_sky_copy = missing_sky.copy()
        missing_under_copy = missing_under.copy()
        same_copy=same.copy()
        
        restos=[]
        for ii in range(len(missing_sky)):
            nn=0
            found=False
            while nn<len(missing_under) and not(found):
                if missing_sky[ii]==missing_under[nn]:
                    same+=[[missing_sky[ii],missing_under[nn]]]
                    del missing_under[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[missing_sky[ii]]

        if restos==[]:
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_under!=[]):
                if len(restos) != len(missing_under):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input(str(missing_under[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_under=missing_under_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_under[t]],[missing_under[t],ok]]
                    del missing_under[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_under):
                    t=0
            op = open('Teams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same

In [28]:
def update_team_names(sky,under,league):
# vamos mudar o nome das dfs de under para o nome das dfs do sky
#assim podemos fazer um merge usando o np.where
    same = file_teams2(sky,under,league)
    same_sky = [same[i][0] for i in range(len(same))]
    same_under = [same[i][1] for i in range(len(same))]
    #HOME
    for i in range(len(same_under)):
        try:
            replaceH=list(np.where(under['HT']==same_under[i])[0])
            home='HT'
        except:
            replaceH=list(np.where(under['HomeTeam']==same_under[i])[0])
            home='HomeTeam'
        for n in replaceH:
            under.at[n,home]=same_sky[i]
    #AWAY
    for i in range(len(same_under)):
        try:
            replaceH=list(np.where(under['AT']==same_under[i])[0])
            away='AT'
        except:
            replaceH=list(np.where(under['AwayTeam']==same_under[i])[0])
            away='AwayTeam'
        for n in replaceH:
            under.at[n,away]=same_sky[i]  
    return sky,under
        
        

In [44]:
a = pd.DataFrame({})
b=pd.DataFrame({'a':[1,2,3],'b':[12,22,23]})
c=pd.concat([a,b])
d=pd.concat([c,c.loc[0]])

In [50]:
pd.Series(c.loc[0])

a     1
b    12
Name: 0, dtype: int64

In [51]:
def to_add_Sky_Under(old_df,new_df):
#df com os jogos que são para adicionar ao ficheiro Sky+Under
    old_teams = list(map(lambda x,y:x+'-'+y,list(old_df['HT']),list(old_df['AT'])))Fmerg
    new_teams = list(map(lambda x,y:x+'-'+y,list(new_df['HT']),list(new_df['AT'])))
    to_add = new_df.loc[np.where(np.isin(new_teams,old_teams,invert=True))]
    old_df = pd.concat([old_df,to_add],sort=False)
    return old_df.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [52]:
def Sky_Download(df): #prepara uma df do Sky a partir do web Scraper
    df = df.drop(['web-scraper-order','web-scraper-start-url','Games','Games-href'],axis=1)
    df['League'] = list(map(lambda x:x.split('.')[-2],list(df['League'])))
    dtime = list(map(lambda x:SkySports_Date(x),list(df['Date'])))
    time = [i[1] for i in dtime]
    date = [i[0] for i in dtime]
    df['Date'] = date
    df['Time'] = time
    colunas = list(df.columns.values)
    df = df[colunas[0:2]+[colunas[-1]]+colunas[2:(len(colunas)-1)]]
    return Sky_leagues(df)

In [59]:
def PortalAdd_Sky(df):
#divide o to_add, nos jogos mesmo para adicionar e os outros
    dix = {}
    remove=[]
    for i in range(len(df)):
        if (df.iloc[i]['AT'] in dix) or (df.iloc[i]['HT'] in dix):
            remove+=[i]
        if df.iloc[i]['HT'] not in dix:
            dix[df.iloc[i]['HT']]=0
        if df.iloc[i]['AT'] not in dix:
            dix[df.iloc[i]['AT']]=0
    df_add = df.drop(remove,axis=0)
    df_notadd = df.loc[remove]
    #df_notadd, são os jogos que ainda não conseguimos prever (dá jeito para saber qnd temos de fzr scrape outra vez)
    return df_add,df_notadd 


In [60]:
def Sky_Desktop_drop(df): 
    df = Sky_leagues(df)
    dtime = list(map(lambda x:SkySports_Date(x),list(df['Date'])))
    time = [i[1] for i in dtime]
    date = [i[0] for i in dtime]
    df['Date'] = date
    df['Time'] = time
    colunas = list(df.columns.values)
    df = df[colunas[0:2]+[colunas[-1]]+colunas[2:(len(colunas)-1)]]
    return df

In [61]:
def Under_drop(df):
    df = Under_leagues(df)
    df = df.drop(['web-scraper-order','web-scraper-start-url','Games','Games-href'],axis=1)
    df['Date'] = list(map(lambda x:UnderStats_Date(x),list(df['Date'])))
    return df

In [66]:
def merge_SU(w): #faz o merge do Sky (S) e do Under (U)
    df = pd.concat(w,sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    i=0
    while i<len(df):
        same = list(np.where(np.logical_and(
        np.logical_and(df['HT']==df.iloc[i]['HT'],df['AT']==df.iloc[i]['AT']),
        np.logical_and(np.array(list(map(lambda x:mergeB_year(x),df['Date'])))==mergeB_year(df.iloc[i]['Date']),
        df['League']==df.iloc[i]['League'])))[0])
        if len(same)==1:
            i+=1
        else:
            del same[0]
            df = df.drop(same,axis=0).reset_index(drop=True)
            #!!!!!!!!!!!!!!!!!!!!!        
            #Se precisar do 25 e do BS, mudar este merge, para tentar ter um q não tenha 'None' nessas odds
            #!!!!!!!!!!!!!!!!!!!!!!
    return df

In [3]:
def get_old_df(league,season):
#vai buscar o ficheiro com os jogos já guardados de uma season específica
    dfs = os.listdir('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//ADF//'+league+'//')
    dfs2 = np.array(list(map(lambda x:x[-7:-5],dfs)))
    try:
        ind = list(np.where(dfs2==season[2:])[0])[0]
    except:
        #se ainda não houver df desta liga desta season, devolve-se uma df vazia
        return pd.DataFrame({})
    return pd.read_excel('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//ADF//'+league+'//'+dfs[ind])

In [2]:
def OldNew_Merge(old_df,new_df):
#Faz o merge da df antiga e da nova e verifica se a informação foi bem "scrapada"
    old_df = old_df.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
    date_old = old_df.iloc[-1]['Date']
    date_new = new_df.iloc[0]['Date']
    if date_new > date_old:
        DF = pd.concat([old_df,new_df],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
        return DF
    else:
        same = list(np.where(np.logical_and(old_df['HT']==new_df.iloc[0]['HT'],old_df['AT']==new_df.iloc[0]['AT']))[0])
        if len(same)!=1:
            raise Exception('Double Game')
        else:
            ind = same[0]
            if (old_df.iloc[ind-1]['FTR'] not in ['H','D','A']) and (ind>0): #assim se for 0 não dá -1 e isso era chato
                raise Exception('Bad Scrape, Games missing')
            old_df_add = old_df[:ind]
            DF = pd.concat([old_df_add,new_df],sort=False).sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)
            return Df
        


In [ ]:
def Predix_Odds(predix,boods): #junta ao ficheiro predix, as odds do Betano_odds
    #Está no Sky_Under
    to_add=pd.DataFrame({})
    dixy=pd.DataFrame({})
    x = boods.columns[-1][-1]
    for p in range(len(predix)):
        same = list(np.where(np.logical_and(
        np.logical_and(boods['HT']==predix.iloc[p]['HT'],boods['AT']==predix.iloc[p]['AT']),
        np.logical_and(np.array(list(map(lambda x,y:days_diff(x,y),list(boods['Date']),[predix.iloc[p]['Date']]*len(list(boods['Date'])))))<=10,
        boods['League']==predix.iloc[p]['League'])))[0])
        if same==[]:
            print(predix.iloc[p]['HT'],' Vs ',predix.iloc[p]['AT'], ' - No ODDS')
            for i in range(1,x+1):
                if i==1:
                    dix = {'ODDH_TOP1':['None'],'Time_1':['None']}
                else:
                    dix.update({'ODDH_TOP'+str(i):['None'],'Time_'+str(i):['None']})
        elif len(same)>1:
            raise Exception('Double Game, Predix_Odds')
        else:
            ss = same[0]
            for i in range(1,x+1):
                if i==1:
                    dix = {'ODDH_TOP1':[predix.iloc[ss]['ODDH_TOP1']],'Time_1':[predix.iloc[ss]['Time_1']]}
                else:
                    dix.update({'ODDH_TOP'+str(i):[predix.iloc[ss]['ODDH_TOP'+str(i)]],'Time_'+str(i):[predix.iloc[ss]['Time_'+str(i)]]})
        dixy = pd.concat([dixy,dix],axis=0,sort=False)           
    predix = pd.concat([predix,dixy],axis=1,sort=False)
    return predix
    
            
    

In [18]:
def input_string_aux(a): #evitar q fique "'qq coisa'"
    if a[0]=="'" and a[-1]=="'":
        a = a[1:-1]
    elif a[0]=='"' and a[-1]=='"':
        a = a[1:-1]
    return a


In [20]:
ok = input_string_aux(input())
ok

'a'


'a'

In [ ]:
def Portal_file_teams(w1,w2,league): #w1 -> df Sky |||| w2 -> df portal
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome portal]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        portal_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
        portal_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
    i = 0
    portal_teams_copy = portal_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(portal_teams) and not(found):
                if sky_teams[ii]==portal_teams[nn]:
                    same+=[[sky_teams[ii],portal_teams[nn]]]
                    del portal_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            portal_teams_copy=portal_teams.copy()
            same_copy=same.copy()
            t=0
            while portal_teams!=[]:
                print('------')
                print(restos)
                ok=input_string_aux(input(str(portal_teams[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    portal_teams=portal_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,portal_teams[t]],[portal_teams[t],ok]]
                    del portal_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(portal_teams):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_portal = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_portal = [portal_teams[i] for i in range(len(portal_teams)) if (portal_teams[i] not in same_portal)]
        missing_sky_copy = missing_sky.copy()
        missing_portal_copy = missing_portal.copy()
        same_copy=same.copy()
        
        restos=[]
        for ii in range(len(missing_sky)):
            nn=0
            found=False
            while nn<len(missing_portal) and not(found):
                if missing_sky[ii]==missing_portal[nn]:
                    same+=[[missing_sky[ii],missing_portal[nn]]]
                    del missing_portal[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[missing_sky[ii]]

        if restos==[]:
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_portal!=[]):
                if len(restos) != len(missing_portal):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input_string_aux(input(str(missing_portal[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_portal=missing_portal_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_portal[t]]]
                    del missing_portal[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_portal):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same

In [ ]:
def Portal_file_teams(w1,w2,league): #w1 -> df Sky |||| w2 -> df portal
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome portal]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        portal_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        try:
            sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
            portal_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
        except:
            try:
                sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
                portal_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
            except:
                sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
                portal_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    i = 0
    portal_teams_copy = portal_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(portal_teams) and not(found):
                if sky_teams[ii]==portal_teams[nn]:
                    same+=[[sky_teams[ii],portal_teams[nn]]]
                    del portal_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            portal_teams_copy=portal_teams.copy()
            same_copy=same.copy()
            t=0
            while portal_teams!=[]:
                print('------')
                print(restos)
                ok=input_string_aux(input(str(portal_teams[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    portal_teams=portal_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,portal_teams[t]]]
                    del portal_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(portal_teams):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_portal = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_portal = [portal_teams[i] for i in range(len(portal_teams)) if (portal_teams[i] not in same_portal)]
        missing_sky_copy = missing_sky.copy()
        missing_portal_copy = missing_portal.copy()
        same_copy=same.copy()
        
        restos=[]
        if (len(missing_sky)==0) and (len(missing_portal)>0):
            t=0
            print(sky_teams)
            time.sleep(0.3)
            while missing_portal!=[]:
                ok = input_string_aux(input(str(missing_portal[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in sky_teams) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    missing_portal=missing_portal_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_portal[t]]]
                    del missing_portal[t]
                if t==len(missing_portal):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
                
                
        elif len(missing_sky)==len(missing_portal):
            for ii in range(len(missing_sky)):
                nn=0
                found=False
                while nn<len(missing_portal) and not(found):
                    if missing_sky[ii]==missing_portal[nn]:
                        same+=[[missing_sky[ii],missing_portal[nn]]]
                        del missing_portal[nn]
                        found=True
                    nn+=1

                if not found:
                    restos+=[missing_sky[ii]]
        else:
            raise Exception('file_teams error. É preciso resolver erro ')
            

        if restos==[]:
            print('What')
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_portal!=[]):
                if len(restos) != len(missing_portal):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input_string_aux(input(str(missing_portal[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_portal=missing_portal_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_portal[t]]]
                    del missing_portal[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_portal):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//PortalTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same

In [ ]:
def Betano_file_teams(w1,w2,league): #w1 -> df Sky |||| w2 -> df betano
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome betano]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        betano_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        try:
            sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
            betano_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
        except:
            try:
                sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
                betano_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
            except:
                sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
                betano_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    i = 0
    betano_teams_copy = betano_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(betano_teams) and not(found):
                if sky_teams[ii]==betano_teams[nn]:
                    same+=[[sky_teams[ii],betano_teams[nn]]]
                    del betano_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            betano_teams_copy=betano_teams.copy()
            same_copy=same.copy()
            t=0
            while betano_teams!=[]:
                print('------')
                print(restos)
                ok=input_string_aux(input(str(betano_teams[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    betano_teams=betano_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,betano_teams[t]]]
                    del betano_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(betano_teams):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_betano = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_betano = [betano_teams[i] for i in range(len(betano_teams)) if (betano_teams[i] not in same_betano)]
        missing_sky_copy = missing_sky.copy()
        missing_betano_copy = missing_betano.copy()
        same_copy=same.copy()
        
        restos=[]
        if (len(missing_sky)==0) and (len(missing_betano)>0):
            t=0
            print(sky_teams)
            time.sleep(0.3)
            while missing_betano!=[]:
                ok = input_string_aux(input(str(missing_betano[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in sky_teams) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    missing_betano=missing_betano_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_betano[t]]]
                    del missing_betano[t]
                if t==len(missing_betano):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
                
                
        elif len(missing_sky)==len(missing_betano):
            for ii in range(len(missing_sky)):
                nn=0
                found=False
                while nn<len(missing_betano) and not(found):
                    if missing_sky[ii]==missing_betano[nn]:
                        same+=[[missing_sky[ii],missing_betano[nn]]]
                        del missing_betano[nn]
                        found=True
                    nn+=1

                if not found:
                    restos+=[missing_sky[ii]]
        else:
            raise Exception('file_teams error. É preciso resolver erro ')
            

        if restos==[]:
            print('What')
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_betano!=[]):
                if len(restos) != len(missing_betano):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input_string_aux(input(str(missing_betano[t])+'? (write "passa" if not sure Or "reset" to restart) '))
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_betano=missing_betano_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_betano[t]]]
                    del missing_betano[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_betano):
                    t=0
            op = open('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same